In [ ]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

import openai
from settings.key import GPT_API_KEY
openai.api_key = GPT_API_KEY



In [ ]:
from LLMSearch.AnswerBot import AnswerBot,parse_answer
from settings.key import GPT_API_KEY, DEEPL_API_KEY
from LLMSearch.GPTQuery import GPTQuery
from LLMSearch.BM25DB import BM25DB
from LLMSearch.SQLTextDB import SQLTextDB
from LLMSearch.Embedding.SBERTFineTuneEmbedding import SBERTFineTuneEmbedding
from LLMSearch.VectorSearcher import VectorSearcher

embedder=SBERTFineTuneEmbedding()
searcher=VectorSearcher(embedder,chunk_size_limit=500)


#bm=BM25DB(initiate=True)
#searcher=SQLTextDB()

# initiate bot module
bot1 = AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=searcher,
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

bot2=AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=SQLTextDB(), 
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

bot3=AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=BM25DB(),
                DEEPL_API_KEY=DEEPL_API_KEY,   
                )

In [ ]:

#bot1.index_documents(initiate=True)

In [ ]:
bot1.index_documents(initiate=False)
bot2.index_documents(initiate=False)
bot3.index_documents(initiate=False)

In [ ]:
query="先生の業績は?"
query="先生の趣味は?"
query="先生は何故自殺をしたのか?"
#query="藤田正博先生の業績は?"
#query="藤田正博先生の業績は?"
query="イオン伝導度を上げるためには?"

query="藤田正博"
query='DLC-BCPsはどのような構造が形成されることがわかりましたか？'
query= 'メタロセン触媒はどのような重合の可能性を与えますか？'

related_documents=[]
#for bot in [bot1,bot2,bot3]:
for bot in [bot1,bot3]:
    related_documents+=bot.search_related_documents(query, k=10)


In [ ]:
import json
import numpy as np

#related documentsから､重複を除く
#related_documents=list(set(related_documents))
#list_of_json = [json.dumps(d, sort_keys=True,cls=MyEncoder) for d in related_documents]

#related_documents=[{"path":d["path"],"text":d["text"],"sim":float(d["sim"])} for d in related_documents]
related_documents=[{"path":d["path"],"text":d["text"],"sim":0} for d in related_documents]
list_of_json = [json.dumps(d, sort_keys=True) for d in related_documents]
related_documents= list(map(json.loads, set(list_of_json)))
print(len(related_documents))
related_documents

In [ ]:
#related_documentを5こずつに分割
related_documents_list=[related_documents[i:i+5] for i in range(0,len(related_documents),5)]
#related_documents_list[1]

In [ ]:
for related_documents in related_documents_list:
    response=bot1.ask(query,context_list= related_documents,stream=True)

    ans=response["answer"]
    for chunk in ans:
        next = chunk['choices'][0]['delta'].get('content', '')
        print(next, end='')

    print("\n")